### Import Libraries

In [ ]:
import os
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from ocslc.switched_linear_mpc import SwitchedLinearMPC


N_STEPS_MIN = 5
N_STEPS_MAX_SS_INT = 100
N_STEPS_MAX = 200
N_STEPS_STEPS = 5

column_names = ["N Steps", "Shooting Method", "Integration Method", "Precompute Time", "Setup Time", "Solve Time"]

path = './csv/'
filename = "solve_times.csv"

In [ ]:
def solve_pannocchia(n_phases, multiple_shooting=True, integration_method='Int'):
    start = time.time()

    model = {
        'A': [np.array([[-0.1, 0, 0], [0, -2, -6.25], [0, 4, 0]])],
        'B': [np.array([[0.25], [2], [0]])],
    }

    n_states = model['A'][0].shape[0]
    n_inputs = model['B'][0].shape[1]

    time_horizon = 10

    x0 = np.array([1.3440, -4.5850, 5.6470])

    swi_lin_mpc = SwitchedLinearMPC(
        model, n_phases, time_horizon, auto=False,
        multiple_shooting=multiple_shooting, x0=x0, propagation=integration_method,
    )

    Q = 1. * np.eye(n_states)
    R = 0.1 * np.eye(n_inputs)
    E = 0. * np.eye(n_states)

    swi_lin_mpc.precompute_matrices(x0, Q, R, E)

    precompute_time = time.time() - start
    start = time.time()

    states_lb = np.array([-100, -100, -100])
    states_ub = np.array([100, 100, 100]) 

    swi_lin_mpc.set_bounds(-1, 1, states_lb, states_ub)

    if swi_lin_mpc.multiple_shooting:
        swi_lin_mpc.multiple_shooting_constraints(x0)

    swi_lin_mpc.set_cost_function(Q, R, x0)

    # Set the initial guess  
    swi_lin_mpc.set_initial_guess(time_horizon, x0)

    swi_lin_mpc.create_solver('ipopt', print_level=0, print_time=False)

    setup_time = time.time() - start
    start = time.time()

    inputs_opt, deltas_opt, _ = swi_lin_mpc.solve()
    solving_time = time.time() - start

    # swi_lin_mpc.plot_optimal_solution(deltas_opt, inputs_opt)
    
    return precompute_time, setup_time, solving_time

### Save the Solving Times

In [ ]:
times = []

for n_steps in range(N_STEPS_MIN, N_STEPS_MAX_SS_INT + 1, N_STEPS_STEPS):
    print(f"n_steps = {n_steps}")
    for multiple_shooting_flag in [True, False]:
        for integration_method in ['Int', 'Exp']:
            shooting_method = "MS" if multiple_shooting_flag else "SS"
            precompute_time, setup_time, solving_time = solve_pannocchia(
                n_steps, multiple_shooting=multiple_shooting_flag, integration_method=integration_method
            )
            times.append([
                n_steps, shooting_method, integration_method,
                precompute_time, setup_time, solving_time,
            ])

### No Single Shooting

In [ ]:
try:
    times
except NameError:
    times = df[column_names].values.tolist()

for n_steps in range(N_STEPS_MAX_SS_INT + N_STEPS_STEPS, N_STEPS_MAX + 1, N_STEPS_STEPS):
    print(f"n_steps = {n_steps}")
    for multiple_shooting_flag in [True, False]:
        for integration_method in ['Int', 'Exp']:
            if multiple_shooting_flag is False and integration_method == 'Int':
                continue
            shooting_method = "MS" if multiple_shooting_flag else "SS"
            precompute_time, setup_time, solving_time = solve_pannocchia(
                n_steps, multiple_shooting=multiple_shooting_flag, integration_method=integration_method
            )
            times.append([
                n_steps, shooting_method, integration_method,
                precompute_time, setup_time, solving_time,
            ])

### Save the Solving Times

In [ ]:
try:
    os.makedirs(path)
except OSError:
    print(f"Creation of the directory {path} failed.")
else:
    print(f"Successfully created the directory {path}.")

df = pd.DataFrame(times, columns=column_names)
out_file = open(path+filename, 'wb')
df.to_csv(out_file)
out_file.close()

### Plot the Results

In [ ]:
from cycler import cycler

default_cycler = (
    cycler(color=[
        '#0072BD', '#D95319', '#EDB120', '#7E2F8E', '#77AC30',
        '#4DBEEE', '#A2142F', '#FF6F00', '#8DFF33', '#33FFF7',
    ]) +
    cycler('linestyle', [
        '-', '--', '-.', ':', '-',
        '--', '-.', ':', '-', '--'
    ])
)

textsize = 16
labelsize = 18

plt.rc('font', family='serif', serif='Times')
plt.rcParams["text.usetex"] = True
plt.rc('text.latex', preamble=r'\usepackage{amsmath} \usepackage{amsfonts} \DeclareMathAlphabet{\mathcal}{OMS}{cmsy}{m}{n}')
plt.rc('xtick', labelsize=textsize)
plt.rc('ytick', labelsize=textsize)
plt.rc('axes', titlesize=labelsize, labelsize=labelsize, prop_cycle=default_cycler)
plt.rc('legend', fontsize=textsize)
plt.rc('grid', linestyle='-.', alpha=0.5)
plt.rc('axes', grid=True)

plt.rcParams['figure.constrained_layout.use'] = True

In [ ]:
# Read data
df = pd.read_csv(path + filename)

# Create figure and subplots with shared x and y axes
fig, axs = plt.subplots(
    2, 2, sharex=True, sharey=True, figsize=(9.6, 7.2),
    # gridspec_kw={'wspace': 0, 'hspace': 0}  # Remove space between subplots
)

for shooting_method in ['SS', 'MS']:
    for integration_method in ['Int', 'Exp']:
        subset = df[
            (df['Shooting Method'] == shooting_method) &
            (df['Integration Method'] == integration_method)
        ]
        
        # Plot into the respective subplots
        axs[0, 0].plot(
            subset['N Steps'], 
            subset['Precompute Time'], 
            label=f'{shooting_method} – {integration_method}'
        )
        axs[0, 1].plot(
            subset['N Steps'], 
            subset['Setup Time'], 
            label=f'{shooting_method} – {integration_method}'
        )
        axs[1, 0].plot(
            subset['N Steps'], 
            subset['Solve Time'], 
            label=f'{shooting_method} – {integration_method}'
        )
        total_time = (
            subset['Precompute Time'] + 
            subset['Setup Time'] + 
            subset['Solve Time']
        )
        axs[1, 1].plot(
            subset['N Steps'], 
            total_time, 
            label=f'{shooting_method} – {integration_method}'
        )

# Set titles for each subplot
axs[0, 0].text(0.5, 0.1, 'Precompute Time', transform=axs[0, 0].transAxes, fontsize=textsize, va='top', ha='center',)
axs[0, 1].text(0.5, 0.1, 'Setup Time',      transform=axs[0, 1].transAxes, fontsize=textsize, va='top', ha='center',)
axs[1, 0].text(0.5, 0.1, 'Solve Time',      transform=axs[1, 0].transAxes, fontsize=textsize, va='top', ha='center',)
axs[1, 1].text(0.5, 0.1, 'Total Time',      transform=axs[1, 1].transAxes, fontsize=textsize, va='top', ha='center',)

axs[0,0].set_xlim(0, N_STEPS_MAX)
axs[0,1].set_xlim(0, N_STEPS_MAX)
axs[1,0].set_xlim(0, N_STEPS_MAX)
axs[1,1].set_xlim(0, N_STEPS_MAX)

# Use log scale for the y-axis
for ax in axs.flat:
    ax.set_yscale('log')

# Label only outer axes
plt.setp(axs[1, :], xlabel='Number of Steps')
plt.setp(axs[:, 0], ylabel='Time [s]')

# Create a single legend for all subplots
handles, labels = axs[0, 0].get_legend_handles_labels()
fig.legend(handles, labels, loc='lower center', ncol=4, bbox_to_anchor=(0.5, -0.05))

plt.tight_layout()
# plt.show()
plt.savefig('./time_vs_steps.pdf', format='pdf', bbox_inches='tight')